In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/workspaces/codespaces-blank/my_paper.pdf")
data=loader.load()

In [2]:
len(data)

15

In [3]:
data[1]

Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-09-14T02:52:38+00:00', 'author': '', 'keywords': '', 'moddate': '2021-09-14T02:52:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'rgid': 'PB:357213035_AS:1103436619751424@1640091199662', 'source': '/workspaces/codespaces-blank/my_paper.pdf', 'total_pages': 15, 'page': 1, 'page_label': '2'}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/354831740\nDevelopment of Multiple Combined Regression Methods for Rainfall\nMeasurement\nChapter · January 2021\nDOI: 10.52458/978-93-91842-08-6-7\nCITATIONS\n0\nREADS\n47\n6 authors, including:\nSome of the authors of this publication are also working on these related projects:\nBangla NLP View project\nImpact learning Implementation View project\nNusrat

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000)
docs=text_splitter.split_documents(data)

print("Total no. of documents: ",len(docs))

Total no. of documents:  44


In [ ]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector=embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [6]:
vectorstore=Chroma.from_documents(documents=docs,embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [7]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":10})

retrieved_docs=retriever.invoke("What is the new development of multiple combined regression for rainfall measurement paper?")

In [8]:
len(retrieved_docs)

10

In [9]:
print(retrieved_docs[5].page_content)

any gauges utilizing these estimations.[3] Besides Prabakaran et al. proposed a
method that speaks to a numerical strategy called Linear Regression to antici-
pate the rainfall in diﬀerent areas in southern states of India.[4] To improvement
Wang et al. showed a case study they proposed an application of generalized
regression neural network (GRNN) model to anticipate yearly precipitation in
Zhengzhou .[5] On the other hand, Sethi et al. presented an exploiting data min-
ing technique for the early prediction of rainfall called multiple linear regression
(MLR) .[6] Sunyoung Lee et al. presented a divide and conquer approach to
predict the rainfall based on the locational information only .[7] Also, Bagirov,
M Adil et al. developed the Clusterwise Linear Regression (CLR) technique for
the prediction of monthly rainfall .[8] In addition, Mohammed Moulana et al.
represented machine learning techniques to precipitation prediction the purpose


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3,max_tokens=500)

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, say that you don't know.
    Use three sentences maximum and keep the answer concise.
    \n\n
    {context}
    """
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [12]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [13]:
response=rag_chain.invoke({"input":"What is the new development of multiple combined regression for rainfall measurement paper?"})
print(response["answer"])

The paper proposes using multiple regression analysis techniques to predict rainfall quantity.  It focuses on finding the best predictive model among various regression methods for real-life applications, particularly for agriculture.  The study involves data collection, preprocessing, model training, and performance analysis.
